## Vinculación con la API del FRED
Se usará la API de la Federal Reserve Economic Data para la importación de un una o más series

In [5]:
import pandas as pd
import requests 

In [6]:
def api_fred(series, key, fechaini, fechafin):
    
    """
    Importar múltiples series de la API del FRED. 
    
    Parámetros
    ----------
    series: dict
        Diccionario de los códigos y nombres de las series.
    
    key: str
        API Key del desarrollador. 
        Se obtiene de: "https://fred.stlouisfed.org/docs/api/api_key.html".
        
        - key: abcdefghijklmnopqrstuvwxyz123456
        
    fechaini: str
        Fecha de inicio de la serie.
        -Diario: yyyy-mm-dd
        -Mensual: yyyy-mm
        -Anual: yyyy
        
    fechfin: str
        Fecha de fin de la serie.
        -Diario: yyyy-mm-dd
        -Mensual: yyyy-mm
        -Anual: yyyy
 
    Retorno: 
    ----------
    df: pd. DataFrame
       Series consultadas
    
    Documentación: 
    --------------
    https://fred.stlouisfed.org/docs/api/fred/
    
    @author: Mauricio Alvarado
             Norbert Andrei Romero Escobedo
    
    """
    
    
    keys = list(series.keys())

    df = pd.DataFrame()
    
    for i in keys:
        url = f"https://api.stlouisfed.org/fred/series/observations?series_id={i}&api_key={key}&file_type=json"
    
        response = requests.get(url) ## HTTP: GET
        
        if response.status_code == 200:
            pass
        else:
            print("Porfavor, revisa los datos ingresados.")
            break
    
        response = response.json()
        observations = response.get("observations")
    
        values_list = []
        time_list = []
        
        for obs in observations:
            values_list.append(float(obs["value"]))
            time_list.append( obs["date"])


        dictio = {
            "time": time_list,
            f"{i}": values_list
        }
        dictio = pd.DataFrame(dictio)
    
        if df.empty is True:
            df = dictio
        else:
            df = pd. merge(df, dictio, how = "outer")

    df.set_index("time", inplace=True)
    df.rename (series, axis = 1, inplace = True)
    return df.loc[fechaini:fechafin]


#search_text = ['MOnetary', 'iNDex']
#api_key = '5ea7806d8a7a82af62865307b8dbf7d0'

def get_codes (search_text, api_key):
    
    """Extear metadatos
    Parámetros
    ----------
    search_text: list
        Consultas
    
    api_key: str
        API Key del desarrollador. 
        Se obtiene de: "https://fred.stlouisfed.org/docs/api/api_key.html".
        
        - key: abcdefghijklmnopqrstuvwxyz123456
    
    Retorno: 
    ----------
    df: pd. DataFrame
       Series consultadas
       
    @author: Mauricio Alvarado
             Norbert Andrei Romero Escobedo
    """
    
    formato = ["+".join(search_text).lower() for i in search_text][0]

    url = f"https://api.stlouisfed.org/fred/series/search?search_text={formato}&api_key={api_key}&file_type=json"
    response = requests.get(url) 
    response.status_code

    response= response.json()['seriess']

    list_id = []
    list_title = []
    list_start = []
    list_end = []
    list_frequency = []
    list_seasonal_adjusment = []

    for i in response:
        list_id.append(i['id'])
        list_title.append(i['title'])
        list_start.append(i['observation_start'])
        list_end.append(i['observation_end'])
        list_frequency.append(i['frequency'])
        list_seasonal_adjusment.append(i['seasonal_adjustment'])


    df = pd.DataFrame({"id":list_id,
                    "title":list_title,
                    "start":list_start,
                    "end": list_end,
                    "frequency": list_frequency,
                    "seasonal_adjusment":list_seasonal_adjusment})


    df.set_index("id", inplace=True) 
    return df



In [7]:
#di = 'CABPPRIVSA'
series_economicas = {
    "INDPRO": "Industrial Production: Total Index ",
    "CPIAUCSL": "Consumer Price Index for All Urban Consumers: All Items in U.S. City Average ",
    "CABPPRIVSA": "New Private Housing Units Authorized by Building Permits for California"
}
api_key = '5ea7806d8a7a82af62865307b8dbf7d0'
fechaini = '1919-01-01'
fechafin = '2000-12-01'

api_fred(series = series_economicas, key = api_key, fechaini = fechaini , fechafin = fechafin)


,Industrial Production: Total Index,Consumer Price Index for All Urban Consumers: All Items in U.S. City Average,New Private Housing Units Authorized by Building Permits for California
time,,,
1919-01-01,4.8773,NaN,NaN
1919-02-01,4.6617,NaN,NaN
1919-03-01,4.5270,NaN,NaN
1919-04-01,4.6078,NaN,NaN
1919-05-01,4.6348,NaN,NaN
...,...,...,...
2000-08-01,92.7900,172.7,12134.908404
2000-09-01,93.1547,173.6,11282.894383
2000-10-01,92.8361,173.9,11611.500512


In [4]:
search_text = ['MOnetary', 'iNDex']
api_key = '5ea7806d8a7a82af62865307b8dbf7d0'

formato = ["+".join(search_text).lower() for i in search_text][0]


url = f"https://api.stlouisfed.org/fred/series/search?search_text={formato}&api_key={api_key}&file_type=json"
response = requests.get(url) 
response.status_code

response= response.json()['seriess']


{'realtime_start': '2023-01-11',
 'realtime_end': '2023-01-11',
 'order_by': 'search_rank',
 'sort_order': 'desc',
 'count': 1703,
 'offset': 0,
 'limit': 1000,
 'seriess': [{'id': 'EPUMONETARY',
   'realtime_start': '2023-01-11',
   'realtime_end': '2023-01-11',
   'title': 'Economic Policy Uncertainty Index: Categorical Index: Monetary policy',
   'observation_start': '1985-01-01',
   'observation_end': '2022-11-01',
   'frequency': 'Monthly',
   'frequency_short': 'M',
   'units': 'Index',
   'units_short': 'Index',
   'seasonal_adjustment': 'Not Seasonally Adjusted',
   'seasonal_adjustment_short': 'NSA',
   'last_updated': '2023-01-02 08:01:01-06',
   'popularity': 27,
   'group_popularity': 27,
   'notes': 'The EPU Categorical Data include a range of sub-indexes based solely on news data. These are derived using results from the Access World News database of over 2,000 US newspapers.\n\nFor further explanation, please see the source page on  Categorical EPU data (https://www.poli

In [6]:
list_id = []
list_title = []
list_start = []
list_end = []
list_frequency = []
list_seasonal_adjusment = []

for i in response:
    list_id.append(i['id'])
    list_title.append(i['title'])
    list_start.append(i['observation_start'])
    list_end.append(i['observation_end'])
    list_frequency.append(i['frequency'])
    list_seasonal_adjusment.append(i['seasonal_adjustment'])


df = pd.DataFrame({"id":list_id,
                   "title":list_title,
                   "start":list_start,
                   "end": list_end,
                   "frequency": list_frequency,
                   "seasonal_adjusment":list_seasonal_adjusment})


df.set_index("id", inplace=True) 
df

,title,start,end,frequency,seasonal_adjusment
id,,,,,
EPUMONETARY,Economic Policy Uncertainty Index: Categorical...,1985-01-01,2022-11-01,Monthly,Not Seasonally Adjusted
MSIMZMP,Monetary Services Index: MZM (preferred),1967-01-01,2013-12-01,Monthly,Seasonally Adjusted
MSIM2,Monetary Services Index: M2 (preferred),1967-01-01,2013-12-01,Monthly,Seasonally Adjusted
MSIALLP,Monetary Services Index: ALL Assets (preferred),1967-01-01,2013-12-01,Monthly,Seasonally Adjusted
OCM2P,Real User Cost Index of MSI-M2 (preferred),1967-01-01,2013-12-01,Monthly,Not Seasonally Adjusted
...,...,...,...,...,...
GNQENEERIX,Nominal Effective Exchange Rate as Based on Co...,2004-01-01,2021-01-01,Annual,Not Seasonally Adjusted
CAFEREERIX,Real Effective Exchange Rate as Based on Consu...,2004-01-01,2021-01-01,Annual,Not Seasonally Adjusted
COGTTTIX,"Terms of Trade for Goods for Congo, Republic of",2004-01-01,2023-01-01,Annual,Not Seasonally Adjusted


In [7]:
url 
print('hola')

hola
